In [123]:
import numpy as np
import pandas as pd
import altair as alt
import datetime as dt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



pio.renderers.default = "plotly_mimetype+notebook_connected"


In [38]:
#| echo = false

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"

In [131]:
## import data (processed by R)

ten_min_day = pd.read_csv('../data/ten_min_day.csv')

ten_min_day.head()

## formatting for later use
ten_min_day['datetime'] = ten_min_day["date"] +"-"+ ten_min_day["time"]
ten_min_day['date']= pd.to_datetime(ten_min_day['date'])
ten_min_day['datetime']= pd.to_datetime(ten_min_day['datetime'])
ten_min_day.info()
ten_min_day.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24192 entries, 0 to 24191
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       24192 non-null  object        
 1   date       24192 non-null  datetime64[ns]
 2   value      24192 non-null  float64       
 3   household  24192 non-null  int64         
 4   phase      24192 non-null  int64         
 5   datetime   24192 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 1.1+ MB


,time,date,value,household,phase,datetime
0,00:00,2012-06-27,0.012044,4,3,2012-06-27 00:00:00
1,00:10,2012-06-27,0.011858,4,3,2012-06-27 00:10:00
2,00:20,2012-06-27,0.011640,4,3,2012-06-27 00:20:00
3,00:30,2012-06-27,0.011608,4,3,2012-06-27 00:30:00
4,00:40,2012-06-27,0.011880,4,3,2012-06-27 00:40:00


## Plotly with dropdown menu and slider
dropdown menu as the households, and slider as the date (first 2 weeks of the data)

In [160]:

# INITIALIZE GRAPH OBJECT
fig = go.Figure(layout=dict(
    legend=dict(groupclick="toggleitem")))

# fig = make_subplots(rows=4, cols=1, vertical_spacing=0.065, shared_xaxes=True)

# ADD TRACES: [c1*y1, c1*y2, c1*y3,..c2*y1, c2*y2,..]
for house in pd.unique(ten_min_day.household):  # 3 household
    for s in pd.unique(ten_min_day.date):  # 14 days

        df1 = ten_min_day.loc[(ten_min_day['household'] == house) & (
            ten_min_day['date'] == s)]
        total_df = df1.loc[(df1['phase'] == 0)]
        p1_df = df1.loc[(df1['phase'] == 1)]
        p2_df = df1.loc[(df1['phase'] == 2)]
        p3_df = df1.loc[(df1['phase'] == 3)]

        fig.add_trace(
            go.Scatter(x=list(total_df.time), y=list(total_df.value), visible=False, name="Total",
                       line=dict(color='red')))
        fig.add_trace(
            go.Scatter(x=list(p1_df.time), y=list(p1_df.value), visible=False, name="Phase 1",
                       line=dict(color='blue')))
        fig.add_trace(
            go.Scatter(x=list(p2_df.time), y=list(p2_df.value), visible=False, name="Phase 2",
                       line=dict(color='green')))
        fig.add_trace(
            go.Scatter(x=list(p3_df.time), y=list(p3_df.value), visible=False, name="Phase 3",
                       line=dict(color='orange')))

        # total 4*14*3 traces

        # fig.add_trace(
        #     go.Scatter(
        #     visible=False,
        #     line=dict(color="#00CED1", width=6),
        #     name="Date : " + str(step),
        #     x=pd.unique(ten_min_day.date),
        #     y=ten_min_day.value))


# Create and add slider
steps = []

# for each year and each continent

for i in range(3*14):
    label = [str(s) for s in ten_min_day['date'].dt.strftime(
        '%Y-%m-%d(%a)').unique()]
    step = dict(
        method="update",
        label=label[i % 14],
        args=[{"visible": [False]*3*14*4},],
        # layout attribute
    )

    step["args"][0]["visible"][i*4] = True
    step["args"][0]["visible"][i*4+1] = True
    step["args"][0]["visible"][i*4+2] = True
    step["args"][0]["visible"][i*4+3] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = []

for i in np.arange(0, 3*14, 14):
    slider = [dict(
        active=0,
        currentvalue={"prefix": "Date: "},
        pad={"t": 50},
        steps=steps[i:i+14],


    )]
    sliders.append(slider)

# INITIALIZE
# MAKE FIRST TRACE VISABLE
fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.update_layout(sliders=sliders[0])


# ADD DROPDOWN TO CHANGE TYPE
# "restyle": modify data or data attributes
# "relayout": modify layout attributes
# "update": modify data and layout attributes
# "animate": start or pause an animation

fig.update_layout(
    title=dict(text="Smart power usage by household",
               font=dict(size=25), yref='paper'),
    legend_title="Phase",
    xaxis_title="Time",
    yaxis_title="Energy consumption, kW.h",
    plot_bgcolor="floralwhite",
    showlegend=True,
    hovermode='x'

)
fig.update_xaxes(tickangle=-45,
                 showspikes=True,
                 spikemode='across',
                 spikesnap='cursor',
                 showline=True,
                 showgrid=True,
                 spikethickness=1,
                
                 )
# IMPORTANT WITH MULTIPLE ARGS YOU NEED EACH IN A DICTIONARY FOR SOME REASON
fig.update_layout(

    updatemenus=[
        dict(
            buttons=list([
                dict(
                    label="Household 04",
                    method="update",
                    args=[{"visible": [False]*0*4+[True]*4+[False]
                           * (3*14-1)*4}, {"sliders": sliders[0]}],


                ),
                dict(
                    label="Household 05",
                    method="update",
                    args=[{"visible": [False]*14*4+[True]*4+[False]
                           * (3*14-1-14)*4}, {"sliders": sliders[1]}],
                ),
                dict(
                    label="Household 06",
                    method="update",
                    args=[{"visible": [False]*28*4+[True]*4+[False]
                           * (3*14-1-14*2)*4}, {"sliders": sliders[2]}],

                )
            ]),
            direction="down",
            showactive=True,
            pad={"r": 10, "t": 10},
            x=0.89,
            xanchor="left",
            y=1.2,
            yanchor="auto",

        )
    ]
)

fig.show()
